In [1]:
import tsinfer
import msprime
import tsdate

/home/savita/DPhil/code/tsbrowse_paper/selective_sweep_example/tsbrowse_selective_sweep_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def make_sweep_ts(n, Ne, L, rho=1e-8, mu=1e-8):
    sweep_model = msprime.SweepGenicSelection(
        position=L/2, start_frequency=0.0001, end_frequency=0.9999, s=0.25, dt=1e-6)
    models = [sweep_model, msprime.StandardCoalescent()]
    ts = msprime.sim_ancestry(
        n, model=models, population_size=Ne, sequence_length=L, recombination_rate=rho, random_seed=1234)
    return msprime.sim_mutations(ts, rate=mu, random_seed=4321)

In [3]:
mu = 1e-8
pop_size = 10_000
rho=1e-8
sim_ts = make_sweep_ts(300, Ne=pop_size, L=5_000_000, mu=mu)
sim_ts.dump("true_topology.trees")

In [ ]:
#tsinfer + tsdate (one iteration of tsinfer, tsdate v 0.2)
i_ts = tsinfer.infer(tsinfer.SampleData.from_tree_sequence(sim_ts))
s_ts = i_ts.simplify()
d_ts = tsdate.date(
    s_ts,
    mutation_rate=mu, max_shape=1000)
d_ts.dump("tsinfer_tsdated.trees")

In [20]:
#Relate inference
import subprocess
import tempfile
import tskit
import numpy as np
def ts_to_haps_sample(ts, haps_output, sample_output, chromosome_number=1, sample_name_field="name"):
    """
    Output the tree sequence as in haps / sample format as required by Relate and ARGneedle
    (see https://mathgen.stats.ox.ac.uk/genetics_software/shapeit/shapeit.html#hapsample)

    ``haps_output`` and ``sample_output`` are the filehandles to which the data will be written.
    To obtain either as strings, you can pass an io.StringIO object here.

    ``sample_name_field`` gives the metadata field in which to look up names to use in the
    output sample file. Where possible, names are taken from the associated individual metadata.
    If samples are not associated with individuals (i.e. this is haploid data), then
    names are taken from node metadata. If no ``sample_name_field`` is present in the metadata,
    the names used are "Individual_N" if samples are associated with individuals, or "Sample_N"
    otherwise.

    Returns an array of the site_ids that were written to the haps file (sites
    with 1 allele or > 2 alleles are skipped)

    .. example::

        with open("out.haps", "wt") as haps, open("out.sample", "wt") as sample:
            ts_to_haps_sample(ts, haps, sample)
    """
    used = np.zeros(ts.num_sites, dtype=bool)
    for v in ts.variants():
        if len(v.alleles) == 1:
            continue
        if len(v.alleles) > 2:
            print(f"Multialleic site ({v.alleles}) at position {v.site.position} ignored")
            continue
        used[v.site.id] = True
        print(
            str(chromosome_number),
            f"SNP{v.site.id}",
            int(v.site.position),
            v.alleles[0],
            v.alleles[1],
            " ".join([str(g) for g in v.genotypes]),
            sep=" ",
            file=haps_output,
        )

    print("ID_1 ID_2 missing", file=sample_output)
    print("0    0    0", file=sample_output)
    individuals = ts.nodes_individual[ts.samples()]
    if np.all(individuals == tskit.NULL):
        # No individuals, just use node metadata
        pass
    else:
        if np.any(individuals == tskit.NULL):
            raise ValueError("Some samples have no individuals")
        _, counts = np.unique(individuals, return_counts=True)
        if np.all(counts == 2):
            if np.any(np.diff(individuals)[0::2]) != 0:
                ValueError("Pairs of adjacent samples must come from the same individual")
        elif np.all(counts == 1):
            pass
        else:
            raise ValueError("Must have all diploid or all haploid samples")
    samples = ts.samples()
    i=0
    while i < len(samples):
        ind1 = ts.node(samples[i]).individual
        ind2 = tskit.NULL
        if ind1 == tskit.NULL:
            try:
                name = ts.node(samples[i]).metadata[sample_name_field].replace(" ", "_")
            except (TypeError, KeyError):
                name = f"Sample_{samples[i]}"
        else:
            try:
                name = ts.individual(ind1).metadata[sample_name_field].replace(" ", "_")
            except (TypeError, KeyError):
                name = f"Individual_{ind1}"
            try:
                ind2 = ts.node(samples[i+1]).individual
            except IndexError:
                pass
        if ind2 == tskit.NULL or ind2 != ind1:
            print(f'{name} NA 0', file=sample_output)
            i += 1
        else:
            print(f'{name} {name} 0', file=sample_output)
            i += 2
    return np.where(used)[0].astype(ts.mutations_site.dtype)

def run_relate(ts, population_size, mu, rho, random_seed=111, path_to_relate="/home/savita/DPhil/code/tsbrowse_paper/Relate/relate_v1.2.2_x86_64_dynamic/"):
    dir = "/home/savita/DPhil/code/tsbrowse_paper/Relate/"
    prefix = "test"
    with open(dir + prefix + ".haps", "wt") as haps, open(dir + prefix + ".sample", "wt") as sample:
        # ts_to_haps_sample routine from https://github.com/tskit-dev/tsconvert/issues/55#issuecomment-1831959994
        ts_to_haps_sample(ts, haps, sample)
    with tempfile.NamedTemporaryFile("wt") as temp:
        cM_per_MB = rho * 1e8
        print("pos", "COMBINED_rate", "Genetic_Map", sep=" ", file=temp)
        print(0, f"{cM_per_MB:.5f}", 0, sep=" ", file=temp)
        print(
            int(ts.sequence_length),
            f"{cM_per_MB:.5f}",
            ts.sequence_length / 1e6 * cM_per_MB,
            sep=" ",
            file=temp)
        temp.flush()

        params = [
            path_to_relate + "bin/Relate",
            "--haps", prefix+".haps",
            "--sample", prefix+".sample",
            "--map", temp.name,
            "-o", "Relate",
            "--mode", "All",
            "-m", f"{mu}",
            "-N", f"{population_size}",
            "--seed", f"{random_seed}",
        ]
        print(f"running `{' '.join(params)}`")
        subprocess.run(params, cwd=dir)

    # Convert to tree sequence format
    params = [
        path_to_relate + "/bin/RelateFileFormats",
        "--mode", "ConvertToTreeSequence",
        "-i", "out",
        "-o", "out",
    ]
    print(f"running `{' '.join(params)}`")
    subprocess.run(params, cwd=dir)
    return tskit.load(dir + "out.trees")

relate_ts = run_relate(sim_ts, pop_size/2, mu, rho)

Multialleic site (('T', 'C', 'A')) at position 1284745.0 ignored
Multialleic site (('C', 'G', 'T')) at position 1307324.0 ignored
Multialleic site (('C', 'G', 'T')) at position 2859263.0 ignored
Multialleic site (('A', 'G', 'C')) at position 3599479.0 ignored
Multialleic site (('A', 'C', 'G')) at position 3757839.0 ignored
Multialleic site (('C', 'T', 'A')) at position 3892870.0 ignored
Multialleic site (('G', 'A', 'T')) at position 3908018.0 ignored
Multialleic site (('G', 'A', 'C')) at position 4071401.0 ignored
Multialleic site (('C', 'G', 'T')) at position 4214239.0 ignored
running `/home/savita/DPhil/code/tsbrowse_paper/Relate/relate_v1.2.2_x86_64_dynamic/bin/Relate --haps test.haps --sample test.sample --map /tmp/tmplbe4sw2i -o out --mode All -m 1e-08 -N 5000.0 --seed 111`



*********************************************************
---------------------------------------------------------
Relate
 * Authors: Leo Speidel, Marie Forest, Sinan Shi, Simon Myers.
 * Doc:     https://myersgroup.github.io/relate
---------------------------------------------------------

---------------------------------------------------------
Using:
  test.haps
  test.sample
  /tmp/tmplbe4sw2i
with mu = 1e-08 and 2Ne = 5000.
---------------------------------------------------------

---------------------------------------------------------
Parsing data..
CPU Time spent: 0.160038s; Max Memory usage: 5.9e+02Mb.
---------------------------------------------------------

---------------------------------------------------------
Read 600 haplotypes with 10659 SNPs per haplotype.
Expected minimum memory usage: 2.8Gb.
---------------------------------------------------------

---------------------------------------------------------
Starting chunk 0 of 0.
------------------------------

running `/home/savita/DPhil/code/tsbrowse_paper/Relate/relate_v1.2.2_x86_64_dynamic//bin/RelateFileFormats --mode ConvertToTreeSequence -i out -o out`


CPU Time spent: 1.148428s; Max Memory usage: 590.228Mb.
---------------------------------------------------------



In [50]:
dir = "/home/savita/DPhil/code/tsbrowse_paper/ARG-Needle/"
prefix = "test"
with open(dir + prefix + ".haps", "wt") as haps, open(dir + prefix + ".sample", "wt") as sample:
    sites = ts_to_haps_sample(ts, haps, sample)
with open(dir + prefix + ".map", "wt") as map, open(dir + prefix + ".demo", "wt") as demo:
    # Make the required mapfile (one line per variant)
    # https://palamaralab.github.io/software/argneedle/manual/#genetic-map-mapmapgz
    # chromosome SNP_name genetic_position_cM physical_position_bp
    for s in sites:
        pos = sim_ts.site(s).position
        print("1", f"Site{s}", f"{pos * rho * 100}", f"{pos}", sep="\t", file=map)
    print("\t".join(["0.0", str(pop_size)]), file=demo)
    print("\t".join(["5000.0", str(pop_size)]), file=demo)
#at this point I was getting errors about numpy version needing to be downgraded, and it persisted, so I ran the below steps from the terminal
#arg_needle --hap_gz test.haps --map test.map --mode sequence --normalize_demography test.demo --out arg_needl
#arg2tskit --arg_path arg_needle.argn --ts_path arg_needle.trees


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/savita/miniconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/savita/miniconda3/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/savita/DPhil/code/tsbrowse_paper/selective_sweep_example/tsbrowse_selective_sweep_env/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/savita/DPhil/code/tsbrowse_paper/selective_sweep_example/tsbrows

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [53]:
sim_ts

In [61]:
#SINGER
#wget https://github.com/popgenmethods/SINGER/tree/main/releases/singer-0.1.8-beta-linux-x86_64
with open("/home/savita/DPhil/code/tsbrowse_paper/SINGER/sim_ts.vcf", "w") as vcf_file:
    sim_ts.write_vcf(vcf_file)
#for the next step I found it easier to run the command from the terminal
